<a id='top'></a>
# Spec Mode Data Calibration: Pipeline Stage 3
---
**Authors**:  
Jo Taylor (jotaylor@stsci.edu), Maria Pena-Guerrero (pena@stsci.edu)

**Latest Update**: 27 May 2021

<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Working with the Spectroscopic Stage 3 Calibration Pipeline, we will:</ul>    
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>
    <li>Look in detail at the steps in common to all spectroscopic modes</li> 
</ul>
</div>

## Table of Contents
* [Introduction](#intro)
* [Spectroscopy Modes](#spec_modes)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#Imports_ID)
* [Convenience Functions](#convenience_functions)
* [Download Data](#download_data)
* [Association Files](#associations)
* [Methods for calling steps/pipelines](#calling_methods)
* [Parameter Reference Files](#parameter_reffiles)
* [calwebb_spec3 - Ensemble calibrations](#spec3) 
   * [Run the entire pipeline](#spec3_at_once)
       * [Using the run() method](#run_method)
       * [Using the call() method](#call_method)
       * [Using the command line](#command_line)
   * [Run the individual pipeline steps](#spec3_step_by_step)
       * [The `Assign Moving Target WCS` step](#assign_mtwcs)
       * [The `Master Background Subtraction` step](#master_background)
       * [The `Exposure to Source` step](#exp_to_source)
       * [The `MIRI MRS Sky Matching` step](#mrs_imatch)
       * [The `Outlier Detection` step](#outlier_detection)
       * [The `Resample` step](#resample_spec)
       * [The `Cube Building` step](#cube_build)
       * [The `1D Extraction` step](#extract_1d)
       * [The `1D Combination` step](#combine_1d)

<a id='intro'></a>
## Introduction

This notebook covers the Stage 3 processing of the spectroscopic mode data calibration module. In this notebook we'll review Stage 3 of the JWST calibration pipeline for spectroscopic data, also known as *calwebb_spec3*. This notebook will not address spectroscopic Time Series Observation (TSO) calibration, which is handled by a separate pipeline, [*calwebb_tso3*](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_tso3.html#calwebb-tso3).

The [*calwebb_spec3* pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_spec3.html) takes one or more calibrated slope images (`*_cal.fits` files) stored in an association file (detailed [below](#associations)) and outputs 1D spectra for all modes, and 2D or 3D spectral products for some modes. The steps performed in order to reach these final products varies with spectroscopic mode. Wide-Field Slitless Spectroscopy (WFSS) and non-TSO SOSS modes are minimally processed by *calwebb_spec3*, while NIRSpec and MIRI modes undergo more correction steps. 

1D extracted spectral products, with suffix *_x1d*, and are produced for all spectroscopic modes. Combined 1D spectral products, with suffix *_c1d* are produced for WFSS and non-TSO SOSS modes. 

All JWST spectroscopy mode data, except TSO SOSS data, are processed through the *calwebb_spec3* pipeline. The pipeline is a wrapper which will string together the appropriate steps for each exposure in the proper order.

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 3 pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_image3) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html)

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/data-processing-and-calibration-files/calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>

The first time you run the pipeline, the CRDS server should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id='Imports_ID'></a>
## Imports

Import packages necessary for this notebook

In [ ]:
# Module with functions to get information about objects:
from glob import glob
import os
import shutil

# Numpy library:
import numpy as np

# To read association file
import json

# To download data
import requests

# To examine parameter reference files
import asdf

# Astropy tools:
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
# %matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [ ]:
# The entire calwebb_spec3 pipeline
from jwst.pipeline import calwebb_spec3

# Individual steps that make up calwebb_spec3
from jwst.assign_mtwcs import AssignMTWcsStep
from jwst.master_background import MasterBackgroundStep
from jwst.exp_to_source import exp_to_source
from jwst.mrs_imatch import MRSIMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.cube_build import CubeBuildStep
from jwst.extract_1d import Extract1dStep
from jwst.combine_1d import Combine1dStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

Check which version of the pipeline we are running:

In [ ]:
import jwst
print(jwst.__version__)

<a id='convenience_functions'></a>
## Define convenience functions and parameters

Here we define some functions that we will use repeatedly throughout the notebook.

In [ ]:
# To make everything easier, all files saved by the pipeline
# and pipeline steps will be saved to the working directory.
# This is important for the level 2 and 3 pipelines
# once we are working with association files.
output_dir = './'

In [ ]:
# Make sure the output directory exists before downloading any data
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

[Top of Notebook](#top)

<a id='download_data'></a>
## Download Data

For this module, we will use calbrated rate files from a NIRCam simulated imaging exposure that is stored in Box. Let's download these files, as well as an association file and some parameter reference files.

In [ ]:
nircam_info = [('https://stsci.box.com/shared/static/p2wlvndw25dk7xwk1tasqevmhkg6p55e.fits',
                'jw98765001001_01101_00001_nrcb5_cal.fits'),
               ('https://stsci.box.com/shared/static/cmhc7kkf5z6373d2vwg7916lrhe5ia7u.fits',
                'jw98765001001_01101_00002_nrcb5_cal.fits'),
               ('https://stsci.box.com/shared/static/sb18cpfqjbw1i09gvw0cpqkj6ymdn899.fits',
                'jw98765001001_01101_00003_nrcb5_cal.fits'),
               ('https://stsci.box.com/shared/static/7d00b9isvss7njhcwmd8uiq8c7s4d845.json',
                'level3_lw_asn.json'),
               ('https://stsci.box.com/shared/static/ja0gkd8c0x8p8konhr84wkuhwnpkqf4s.asdf',
                'jwst_nircam_pars-tweakregstep_0006.asdf'),
               ('https://stsci.box.com/shared/static/yahdw55fotwrh7i6hhcxksj97qkf7j4r.asdf',
                'nircam_pars-sourcecatalogstep_f444w_clear.asdf')
              ]
nircam_files = download_files(nircam_info, output_dir)

You can also download the example MIRI files if you wish to try the exercise of running them through the pipeline.

In [ ]:
miri_info = [('https://stsci.box.com/shared/static/0fbtsmehvlfnw5zrnzul0yc2f0l6i16r.json',
              'miri_level3_asn.json'),
             ('https://stsci.box.com/shared/static/te5dmv32nu09u2bnxmzfcsanaibjp36z.fits',
              'miri_F770W_exp1_cal.fits'),
             ('https://stsci.box.com/shared/static/qmzwrn0nq3m7m72kw2c3ajaqvxv0wndu.fits',
              'miri_F770W_exp2_cal.fits'),
             ('https://stsci.box.com/shared/static/yo9b6i5i0j1kz3nxeqap25ohyzm26omo.fits',
              'miri_F770W_exp3_cal.fits')
            ]
miri_files = download_files(miri_info, output_dir)

[Top of Notebook](#top)

<a id='associations'></a>
## Association Files

The Stage 3 pipeline must be called using a json-formatted file called an ["association" file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/associations/index.html). When retrieving your observations from MAST, you will be able to download the association files for your data along with the fits files containing the observations.

The association file presents your data files in organized groups. Let's open the level 3 association file for the data to be processed in this notebook and look at its contents.

In [ ]:
asn_file = os.path.join(output_dir, 'level3_lw_asn.json')

In [ ]:
# Open the association file and load into a json object
with open(asn_file) as f_obj:
  asn_data = json.load(f_obj)

In [ ]:
asn_data

Here we see that the association file begins with a few lines of data that give high-level information about the association. The most important entry here is the `asn_rule` field. Association files have different formats for the different stages of the pipeline. You should be sure that the `asn_rule` matches the pipeline that you will be running. In this case we'll be running the Stage 3 pipeline, and we see that the `asn_rule` mentions "Level3", which is what we want.

Beneath these lines, we see the `products` field. This field contains a list of dictionaries that specify the files that belong to this association, and the types of those files. When the Stage 3 pipeline is run on this association file, all files listed here will be run through the calibration steps.

<a id='diy_association'></a>
##### Creating your own association files

As we will see below, when calling the steps of the pipeline one at a time, you can provide each step with either the output object from the preceding step, or you can provide an association file. Generally the former is easier to do. 

Since each step outputs modified observation files, in order to provide an association file for each step, you'll have to make a new association file for each step that lists the new observation files. Here we will show an example of how to create a new association file for the Stage 3 pipeline or pipeline steps. Keep in mind that you may also simply make a copy of an existing association file and update the member filenames.

We'll use the [`asn_from_list()` function](https://jwst-pipeline.readthedocs.io/en/stable/api/jwst.associations.asn_from_list.asn_from_list.html#jwst.associations.asn_from_list.asn_from_list) to create the new association file designed to be used in the `skymatch` step using as input the outputs from the `tweakreg` step. As input, we need the list of member files, as well as the product name. The product name will be prepended onto the output filenames by the pipeline or step that uses this association file.

In [ ]:
tweak_files = ['level3_lw_asn_0_tweakregstep.fits',
               'level3_lw_asn_1_tweakregstep.fits',
               'level3_lw_asn_2_tweakregstep.fits']
tweak_product = 'manual_asn_file'

In [ ]:
tweakreg_asn = asn_from_list.asn_from_list(tweak_files, rule=DMS_Level3_Base, product_name=tweak_product)

Here is our new association, containing the three files created by the `tweakreg` step.

In [ ]:
tweakreg_asn

Now save the new association to a json file.

In [ ]:
output_test = 'manual_tweakreg_asn.json'
with open(output_test, 'w') as outfile:
    name, serialized = tweakreg_asn.dump(format='json')
    outfile.write(serialized)

Another way to create an association file is to use the [asn_from_list command line tool](https://jwst-pipeline.readthedocs.io/en/stable/api/jwst.associations.asn_from_list.asn_from_list.html#jwst.associations.asn_from_list.asn_from_list). The same association file that we created above can be created via the following command on the command line. **NOTE:** The command below will not work correctly until we have run the Tweakreg step and created some `*_tweakregstep.fits` files.

<div class="alert alert-block alert-info">
    
```
asn_from_list -o manual_tweakreg_asn.json --product-name manual_asn_file level3_lw_asn_?_tweakregstep.fits
```  
</div>

You can see how creating a new association file for each step of the pipeline would become cumbersome. For the step-by-step calls to the pipeline below, we will input the preceding step's output object into each step.

[Top of Notebook](#top)

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Within this notebook, in the section where we [call the entire pipeline](#image3_at_once), as well as the section where we [call the WCS Refinement](#tweakreg_call) step, we show examples of all three methods. For the remainder of the pipeline steps, we will focus on using the `run()` method.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Currently, CRDS contains a default version of a parameter reference file for the [WCS refinement](#tweakreg) step only. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

Let's take a look at the contents of a parameter reference file. NIRCam does not currently have a parameter reference file for the Stage 3 imaging pipeline, so we'll look at the parameter reference file for the WCS refinement step. We'll open it using the asdf package, and use the `tree` attribute to see what's inside:

In [ ]:
tweak_param_reffile = 'jwst_nircam_pars-tweakregstep_0006.asdf'

In [ ]:
tweak_params = asdf.open(tweak_param_reffile)

In [ ]:
tweak_params.tree

The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Tweakreg` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

In [ ]:
# Don't forget to close the file
tweak_params.close()

[Top of Notebook](#top)

---
<a id='spec3'></a>
## The calwebb_spec3 pipeline: Ensemble processing

In the sections below, we will run the Stage 3 pipeline using an association file containing several NIRCam exposures. We will first call the entire *calwebb_spec3* pipeline itself. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order.

After running the entire pipeline, we will go back to the original calibrated slope images and manually run them through each of the steps that comprise the Stage 3 pipeline. For each step we will describe in more detail what is going on and examine how the exposure files have changed.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_spec3) on the calwebb_spec3 algorithm page for a map of the steps are performed on the input data.

<a id='image3_at_once'></a>
### Run the entire `calwebb_spec3` pipeline

In this section we show how to run the entire calwebb_spec3 pipeline with a single call. 

We show all three methods for calling the pipeline.


We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.


<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several paramaters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps compirising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

Let's look at the available parameters for the tweakreg step and the source catalog step, and manually set some of these in our call to `run()`.

In [ ]:
print(TweakRegStep.spec)

In [ ]:
print(SourceCatalogStep.spec)

<a id='detector1_using_run'></a>

<div class="alert alert-block alert-info">
Finally, let's run the pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [ ]:
# Create an instance of the pipeline class
spec3 = calwebb_spec3.Spec3Pipeline()

# Set some parameters that pertain to the
# entire pipeline
spec3.output_dir = output_dir
spec3.save_results = True

# Set some parameters that pertain to some of
# the individual steps

# Call the run() method
spec3.run(asn_file)

This cell will take a few minutes to run. While we're waiting, let's look at the other two methods that can be used to call the pipeline. Then we'll come back here and look at the log meessages output by this cell so we can see what happened.

<a id='call_method'></a>
#### Call the pipelne using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method. In this case, there is no default parameter reference file in CRDS. Therefore, when using the `call()` method, we won't specify a paramter reference file, and the pipeline will fall back to using default parameter values that are defined in the code.

Since we just ran the pipeline with the `run()` method above, we won't actually execute the call to `call()`. But if you wish to try it out, use the pull-down menu above to change the cell to be 'Code', and then execute it. (Or, Click 'Cell' > 'Cell Type' > 'Code')

<div class="alert alert-block alert-info">

<b>Method #1:</b>
Provide the name of the association file, and set pipeline-specific input paramters. Here, we don't set any step-specific paramters, so the pipeline will fall back to using default values for everything.
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

<a id='command_line'></a>
#### Call the pipeline from the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

Since NIRCam does not have a parameter reference file for calwebb_image3, we show how to use the `strun` command and manually update parameter values. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
We provide the name of the pipeline class, the observation file, and explicitly set pipeline- and step-specific parameters. You can see that the command quickly becomes quite large with the added parameter settings. 
    
```
    strun jwst.pipeline.Image3Pipeline level3_lw_asn.json --save_results=True --output_dir='./' --steps.tweakreg.kernel_fwhm=2.302 --steps.tweakreg.snr_threshold=10. --steps.tweakreg.brightest=20 --steps.source_catalog.kernel_fwhm=2.302 --steps.source_catalog.snr_threshold=10.
```
</div>

### Examine the outputs

Get the input filenames from the association file

In [ ]:
input_files = [item['expname'] for item in asn_data['products'][0]['members']]       

In [ ]:
input_files

Define the names of the other output files.

In [ ]:
mosaic_file = os.path.join(output_dir, 'l3_lw_results_i2d.fits')
source_cat_file = os.path.join(output_dir, 'l3_lw_results_cat.ecsv')
segmentation_map_file = os.path.join(output_dir, 'l3_lw_results_segm.fits')
cr_flagged_files = [item.replace('cal.fits', 'crf.fits') for item in input_files]

Read in the final mosaic image and display

In [ ]:
mosaic = datamodels.open(mosaic_file)

In [ ]:
show_image(mosaic.data, vmin=0, vmax=5)

Let's look at the segmentation map that was created by the `source_catalog` step. This shows which pixels are associated with the identified sources.

In [ ]:
seg_map = fits.getdata(segmentation_map_file)

In [ ]:
show_image(seg_map, vmin=0, vmax=5, scale='linear')

And now examine the actual source catalog. For each source, the catalog lists the location, along with flux and AB/Vega magnitude values in three different apertures, as well as calculated values for an infinite aperture. Within the documentation, you can see the [full list of column definitions](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html#source-catalog-table).

In [ ]:
source_cat = ascii.read(source_cat_file)

In [ ]:
source_cat

Finally, let's overlay the source catalog on top of the mosaic image. In order to cut down on the number of spurious detections, we only show sources above a minimum flux limit. Another way to cut down on the number of spurious detections would be to change some of the `source_catalog` parameter values when calling the pipeline above.

In [ ]:
overlay_catalog(mosaic.data, source_cat, flux_limit=5e-7, vmin=0, vmax=10,
                title='Final mosaic with source catalog')

[Top of Notebook](#top)

<a id='image3_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we run the steps contained within calwebb_spec3 one at a time, in order to more clearly see what each step is doing.

<a id='assign_mtwcs'></a>
### The `Assign Moving Target WCS` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used

[Top of Notebook](#top)

<a id='master_background'></a>
## The `Master Background Subtraction` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used

[Top of Notebook](#top)

<a id='exp_to_source'> </a>
## The `Exposure to Source` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used


[Top of Notebook](#top)

<a id='mrs_imatch'> </a>
## The `MIRI MRS Sky Matching` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used

[Top of Notebook](#top)

<a id='outlier_detection'></a>
## The `Outlier Detection` step

#### Summary

This step uses the collection of input files to identify and flag any cosmic rays or other transient image artifacts that were not flagged by the jump step in calwebb_detector1. While the jump step looked for large pixel-based deviations in the signal from group-to-group within an integration, the outlier detection step looks for large sky-based exposure-to-exposure devations in the signal. If a given location on the sky shows no signal above the noise in 4 out of 5 exposures, but a bright source in the remaining exposure, the outlier detction step will flag in the DQ map the pixels containing the source in the fifth exposure. These pixels will be ignored when the exposures are later combined into a final mosaic image.

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/main.html) of the step.

#### Arguments

There are [numerous optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/arguments.html) for this step, including several that apply to the resample step, which this step makes use of.


#### Reference files used

This step does not use any reference files.

[Top of Notebook](#top)

<a id='resample_spec'> </a>
## The `Resample` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments

There is a list of [optional Astrodrizzle-style](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html) input parameters that can be used to customize the resampling process.

#### Reference files used

This step uses the [`DRIZPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file. This file contains Astrodrizzle-style keywords that can be used to control the details of the resampling.

[Top of Notebook](#top)

<a id='cube_build'> </a>
## The `Cube Building` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used

[Top of Notebook](#top)

<a id='extract_1d'> </a>
## The `1D Extraction` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used

#### Parameter reference files


[Top of Notebook](#top)

<a id='combine_1d'> </a>
## The `1D Combination` step

#### Summary

#### Affected Modes
This step is run for the following modes:
- NIRSpec:
- MIRI:
- NIRISS:
- NIRCam:

#### Documentation

#### Arguments

#### Reference files used


[Top of Notebook](#top)